## Assignment 2
#### Group Members:
- Abhijit Anil (aa94375)
- Anubhav Nehru (an33868)
- Antra Sinha (as224589)
- Avi Walyat(aw39578)
- Gaytri Riya Vasal(grv377)

#### Problem Description


In [ ]:
# Use this code if using Colab to run Selenium

# Make sure to go to Runtime -> Change runtime and set GPU as hardware accelerator

# !kill -9 -1 # Use this line to delete this VM and start a new one.
# The above line deletes all files and folders from the current VM and allocates a new one.

#Selenium is an open-source tool that automates web browsers.
!pip install selenium
!apt-get -q update   #Used to handle installation and removal of softwares and libraries
!apt install -yq chromium-chromedriver #ChromeDriver is a separate executable that Selenium WebDriver uses to control Chrome.
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
import pandas as pd
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
#WebDriver is a browser automation framework that works with open source APIs.
#The framework operates by accepting commands, sending those commands to a browser, and interacting with applications.
chrome_options = webdriver.ChromeOptions()
#headless means running chrome with chrome.exe
chrome_options.add_argument('--headless')

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
wd = webdriver.Chrome(options=chrome_options)

wd.get("https://www.acclaimedmusic.net/year/alltime_songs.htm")

In [ ]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

rank = []
artist = []
song = []
genre = []
year =[]

for i in range(2,10000):

  rank_row = wd.find_element(by=By.XPATH, value="/html/body/div[2]/div[2]/table/tbody/tr["+str(i)+"]/td[1]")
  artist_row = wd.find_element(by=By.XPATH,value="/html/body/div[2]/div[2]/table/tbody/tr["+str(i)+"]/td[2]")
  song_row = wd.find_element(by=By.XPATH, value="/html/body/div[2]/div[2]/table/tbody/tr["+str(i)+"]/td[3]")
  genre_row = wd.find_element(by=By.XPATH, value="/html/body/div[2]/div[2]/table/tbody/tr["+str(i)+"]/td[4]")
  year_row = wd.find_element(by=By.XPATH, value="/html/body/div[2]/div[2]/table/tbody/tr["+str(i)+"]/td[5]")


  rank.append(rank_row.text)
  artist.append(artist_row.text)
  song.append(song_row.text)``
  year.append(year_row.get_attribute("textContent").replace('\n', ''))
  genre.append(genre_row.get_attribute("textContent").replace('\n', ''))


In [ ]:
music_table1 = pd.DataFrame({'Rank': rank, 'Artist': artist, 'Song': song,  'Genre': genre, 'Year': year,})

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import re
import pandas as pd

In [ ]:
hot100_df = music_table1
hot100_df_10 = hot100_df[:4000]

In [ ]:
# delay after each execution of call for not exceeding the requests count and also not to overburden the server
delay = 5


lyrics_list = []
for index, song in hot100_df_10.iterrows():
    cleaned_song = re.sub(r'[^a-zA-Z]', '', song['Song'].replace(' ', '').lower())
    cleaned_artist = re.sub(r'[^a-zA-Z]', '', song['Artist'].replace(' ', '').lower())
    final_url = "https://www.azlyrics.com/lyrics/"+cleaned_artist+"/"+cleaned_song+".html"

    try:
        html_page = urlopen(final_url)
        soup = BeautifulSoup(html_page, 'html.parser')

        html_pointer = soup.find('div', attrs={'class':'ringtone'})
        song_name = html_pointer.find_next('b').contents[0].strip()
        lyrics = html_pointer.find_next('div').text.strip()

        lyrics_list.append(lyrics)
        
        print("Lyrics successfully written to file for : " + str(song['Song']))
        
    except:
        print("Lyrics not found for : " + str(song['Song']))
        # lyrics_not_found_for.append(song)
        lyrics_list.append(' ')
        
    finally:
        sleep(delay)

In [ ]:
hot100_df_10['Lyrics'] = lyrics_list

In [ ]:
hot100_df_10.to_csv('final_song_lyric_data.csv', index=False) 

Above code for scraping

In [1]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
from spacy.lang.en import English
nlp = English()
nlp.max_length = 10000000
import pandas as pd
nltk.download('punkt')
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

%load_ext autoreload
%autoreload 2
import warnings
import nltk
from tqdm import tqdm
warnings.filterwarnings('ignore')
#execute the following of not downloaded already:
nltk.download('wordnet')
nltk.download('stopwords')
# Download the VADER lexicon for sentiment analysis
nltk.download('vader_lexicon')
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
from nltk.corpus import stopwords
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pyLDAvis
from collections import Counter
import pyLDAvis.gensim
from nltk import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abhijit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abhijit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Abhijit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abhijit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Abhijit\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Function to preprocess text
def preprocess(text):
        text = re.sub(r'\[.*?\]', '', text)
        text = re.sub(r'\\[\'"nrt]', '', text)
        text = text.replace('\n', ' ')
        text = re.sub(r"'", '', text)
        text = re.sub(r"[^a-zA-Z0-9\s]", '', text)
        return text


def clean_lyrics(lyrics):
    # Remove terms starting with Translations
    lyrics = re.sub(r"Translations\S*", " ", lyrics)

    # Remove text between square brackets
    lyrics = re.sub(r"\[.*?\]", " ", lyrics)

    # Remove any numbers followed by 'Embed'
    lyrics = re.sub(r"\d+Embed", " ", lyrics)

    # Remove new line characters
    lyrics = lyrics.replace('\n', ' ')

    # Convert all text to lowercase
    lyrics = lyrics.lower()

    # Remove specific text if it's followed by 'you might also like'
    lyrics = re.sub(
        r"see(?!.*see.*you might also like).*you might also like", "", lyrics
    )

    # Return the cleaned lyrics
    return lyrics

In [3]:
# Create a SentimentIntensityAnalyzer object
sid = SentimentIntensityAnalyzer()

In [4]:
df = pd.read_csv('final_song_lyric_data.csv')
df

,Rank,Artist,Song,Genre,Year,Lyrics
0,1,Bob Dylan,Like a Rolling Stone,"Folk Rock,Singer/Songwriter",1965,Once upon a time you dressed so fine\nYou thre...
1,2,Nirvana,Smells Like Teen Spirit,Grunge,1991,"Load up on guns, bring your friends\nIt's fun ..."
2,3,The Beatles,A Day in the Life,"Art Rock,Progressive Pop,Psychedelic Pop",1967,
3,4,The Beach Boys,Good Vibrations,"Baroque Pop,Progressive Pop,Psychedelic Pop,Su...",1966,
4,5,Chuck Berry,Johnny B. Goode,Rock & Roll,1958,Deep down in Louisiana close to New Orleans\nW...
...,...,...,...,...,...,...
3995,3996,Janis Ian,Society's Child (Baby I've Been Thinking),"Folk Pop,Singer/Songwriter",1966,
3996,3997,Cat Stevens,Wild World,"Folk Pop,Singer/Songwriter",1970,Now that I've lost everything to you\nYou say ...
3997,3998,Crowded House,Weather with You,Pop Rock,1991,Walking 'round the room singing Stormy Weather...
3998,3999,YG feat. Nipsey Hussle,FDT (Fuck Donald Trump),"Hardcore Hip Hop,Political Hip Hop,West Coast ...",2016,


In [5]:
df_cleaned = df[df['Lyrics'] != ' ']
df_cleaned['Decade'] = (df_cleaned['Year'] // 10) * 10
df_cleaned = df_cleaned[df_cleaned['Decade'] >= 1960]

In [6]:
df_cleaned['Lyrics_Clean'] = lyric_keywords = [preprocess(x) for x in df_cleaned['Lyrics']]

df_cleaned['Lyrics_Clean'] = lyric_keywords = [clean_lyrics(x) for x in df_cleaned['Lyrics_Clean']]

In [7]:
df_cleaned

,Rank,Artist,Song,Genre,Year,Lyrics,Decade,Lyrics_Clean
0,1,Bob Dylan,Like a Rolling Stone,"Folk Rock,Singer/Songwriter",1965,Once upon a time you dressed so fine\nYou thre...,1960,once upon a time you dressed so fine you threw...
1,2,Nirvana,Smells Like Teen Spirit,Grunge,1991,"Load up on guns, bring your friends\nIt's fun ...",1990,load up on guns bring your friends its fun to ...
5,6,Aretha Franklin,Respect,Southern Soul,1967,"What you want\nBaby, I got it\nWhat you need\n...",1960,what you want baby i got it what you need do y...
7,8,Marvin Gaye,What's Going On,Soul,1971,"Hey, hey-hey\nHey, what's happenin'?\nHey, bro...",1970,hey heyhey hey whats happenin hey brother what...
8,9,Marvin Gaye,I Heard It Through the Grapevine,Motown Sound,1968,I bet you're wonderin' how I knew\n'Bout your ...,1960,i bet youre wonderin how i knew bout your plan...
...,...,...,...,...,...,...,...,...
3987,3988,Sean Paul,Get Busy,Ragga,2002,Shake that thing Miss Kana Kana\nShake that th...,2000,shake that thing miss kana kana shake that thi...
3991,3992,PJ Harvey,Dress,"Alternative Rock,Singer/Songwriter",1991,Put on that dress\nI'm going out dancing\nStar...,1990,put on that dress im going out dancing startin...
3992,3993,Jackson C. Frank,Blues Run the Game,"Contemporary Folk,Singer/Songwriter",1965,"Catch a boat to England, baby\nMaybe to Spain\...",1960,catch a boat to england baby maybe to spain wh...
3996,3997,Cat Stevens,Wild World,"Folk Pop,Singer/Songwriter",1970,Now that I've lost everything to you\nYou say ...,1970,now that ive lost everything to you you say yo...


In [8]:
all_words = []
# Assuming 'Word_List' is the column containing lists of words in your DataFrame
# for i in df_cleaned['Word_List']:
for i in df_cleaned['Lyrics_Clean']:
    words = i.lower().split()
    all_words.extend(words)
    # all_words.extend(i)

# Calculate word frequencies
word_freq = Counter(all_words)
# Get the top N most common words (adjust N as needed)
N = 100
top_N_words = word_freq.most_common(N)

# Create a DataFrame with two columns
top_100_df = pd.DataFrame(top_N_words, columns=['Word', 'Freq'])
print(top_100_df.head(20))

    Word   Freq
0    the  22825
1      i  19361
2    you  19242
3    and  13045
4     to  12573
5      a  11599
6     me   9105
7     it   7894
8     my   7539
9     in   7217
10    on   5953
11    of   5723
12  your   5534
13    im   5161
14  that   4810
15  dont   4177
16   all   4076
17    is   3952
18   for   3941
19    be   3667


In [9]:
stop_words = set(stopwords.words('english'))
custom_stopwords = {"nt","oh","na","go","ca", "ai","ooh","wan","wo","la","ah","ta","ya","put","gon","let","say","way","got","le","le","babi","gui","ey","im","que","le","bamba","oi","ba","les",'ooh','yeah','hey','whoa','woah', 'ohh', 'was', 'mmm', 'oooh','yah','yeh','mmm', 'hmm','deh','doh','jah','wa'}  # Replace with your own words
stop_words.update(custom_stopwords)

In [10]:
def lyric_preprocessing(comment) :

    # Converting coomments to lower case, tokenizing the comments and removing punctuation from a comments
    words = [i for i in word_tokenize(comment.lower())]
    words = [word for word in words if word not in stop_words and word]
    return (words,' '.join(words))

In [11]:
# Initialize empty lists to collect the results
word_lists = []
cleaned_sentences = []

for comment in df_cleaned['Lyrics_Clean']:
    if pd.notna(comment) and isinstance(comment, str): 
        output = lyric_preprocessing(comment)
        word_lists.append(output[0])
        cleaned_sentences.append(output[1])

# After processing all comments, assign the lists to DataFrame columns
df_cleaned['Word_List'] = word_lists
df_cleaned['Cleaned_Sentence'] = cleaned_sentences

In [12]:
# lyric_corpus = df_cleaned['Lyrics_Clean']
lyric_corpus = df_cleaned['Cleaned_Sentence']

In [13]:
lyric_corpus_tokenized = []
tokenizer = RegexpTokenizer(r'\w+')
for lyric in tqdm(lyric_corpus):
    tokenized_lyric = tokenizer.tokenize(lyric.lower())  #tokenize and lower each lyric
    lyric_corpus_tokenized.append(tokenized_lyric)

100%|██████████| 2029/2029 [00:00<00:00, 16454.83it/s]


In [14]:
def count_token_in_corpus (corpus): #returns number of tokens for any given corpus
    return_count = 0
    for song in corpus:
        return_count += len(song)
    print("Total of", return_count, "tokens in the current corpus.")

In [15]:
count_token_in_corpus(lyric_corpus_tokenized)

Total of 281134 tokens in the current corpus.


In [16]:
for s,song in tqdm(enumerate(lyric_corpus_tokenized)):
    filtered_song = []    
    for token in song:
        if len(token) > 2 and not token.isnumeric():
            filtered_song.append(token)
    lyric_corpus_tokenized[s] = filtered_song

2029it [00:00, 35528.33it/s]


In [17]:
count_token_in_corpus(lyric_corpus_tokenized)

Total of 276410 tokens in the current corpus.


In [18]:
lemmatizer = WordNetLemmatizer()

for s,song in tqdm(enumerate(lyric_corpus_tokenized)):
    lemmatized_tokens = []
    for token in song:
        lemmatized_tokens.append(lemmatizer.lemmatize(token))
    lyric_corpus_tokenized[s] = lemmatized_tokens

2029it [00:03, 623.30it/s] 


In [19]:
count_token_in_corpus(lyric_corpus_tokenized)

Total of 276410 tokens in the current corpus.


In [20]:
from collections import Counter
all_words = []
# Assuming 'Word_List' is the column containing lists of words in your DataFrame
for i in df_cleaned['Cleaned_Sentence']:
    words = i.lower().split()
    all_words.extend(words)
    # all_words.extend(i)

# Calculate word frequencies
word_freq = Counter(all_words)

In [21]:
add_stop_words = [word for word, count in word_freq.items() if count > 1000]

In [22]:
stop_words.update(add_stop_words)


for s,song in tqdm(enumerate(lyric_corpus_tokenized)):
    filtered_text = []    
    for token in song:
        if token not in stop_words:
            filtered_text.append(token)
    lyric_corpus_tokenized[s] = filtered_text

2029it [00:00, 69163.10it/s]


In [23]:
count_token_in_corpus(lyric_corpus_tokenized)

Total of 217688 tokens in the current corpus.


In [24]:
dictionary = Dictionary(lyric_corpus_tokenized)
print('Number of unique tokens: ', len(dictionary))

Number of unique tokens:  16169


In [25]:
dictionary.filter_extremes(no_below = 10, no_above = 0.8)
print('Number of unique tokens: ', len(dictionary))

Number of unique tokens:  1882


In [26]:
from gensim.corpora import MmCorpus

gensim_corpus = [dictionary.doc2bow(song) for song in lyric_corpus_tokenized]

#create index to dictionary
temp = dictionary[0]  # "loads" the dictionary
id2word = dictionary.id2token

In [27]:
# Specify the number of topics

num_topics = 20

# Train the LDA model
lda_model = LdaModel(gensim_corpus, num_topics=num_topics, id2word=dictionary, passes=15, random_state=30)


# Get the topic distribution for each song
song_topic_distribution = [lda_model[doc] for doc in gensim_corpus]

In [28]:
lda_model.print_topics()

[(0,
  '0.177*"sing" + 0.101*"babe" + 0.083*"enough" + 0.040*"nothing" + 0.035*"darling" + 0.031*"girl" + 0.029*"thing" + 0.026*"youve" + 0.021*"bring" + 0.021*"wont"'),
 (1,
  '0.028*"dance" + 0.019*"moment" + 0.016*"soon" + 0.016*"stop" + 0.015*"big" + 0.014*"tryin" + 0.014*"name" + 0.013*"afford" + 0.013*"next" + 0.013*"sweet"'),
 (2,
  '0.103*"tonight" + 0.084*"light" + 0.062*"find" + 0.056*"move" + 0.050*"golden" + 0.031*"heart" + 0.028*"prove" + 0.026*"hand" + 0.023*"shine" + 0.022*"neon"'),
 (3,
  '0.128*"arm" + 0.095*"shame" + 0.079*"high" + 0.076*"live" + 0.060*"wrapped" + 0.036*"pain" + 0.031*"lose" + 0.026*"blame" + 0.025*"wouldnt" + 0.024*"stay"'),
 (4,
  '0.132*"end" + 0.101*"radio" + 0.087*"remember" + 0.036*"need" + 0.034*"dream" + 0.028*"sound" + 0.025*"century" + 0.024*"part" + 0.023*"fast" + 0.022*"past"'),
 (5,
  '0.083*"higher" + 0.076*"gone" + 0.062*"electric" + 0.052*"hold" + 0.045*"wherever" + 0.041*"far" + 0.039*"black" + 0.032*"life" + 0.031*"hole" + 0.029*"wan

In [29]:
# Extract topic-word distribution
topic_word_dist = lda_model.show_topics()

topic_names = {
    'Topic 1': 'Love',
    'Topic 2': 'Dance',
    'Topic 3': 'Nightlife',
    'Topic 4': 'Struggles',
    'Topic 5': 'Memories',
    'Topic 6': 'Aspiration',
    'Topic 7': 'World',
    'Topic 8': 'Identity',
    'Topic 9': 'Rock',
    'Topic 10': 'Relationships',
    'Topic 11': 'Desires',
    'Topic 12': 'Reality',
    'Topic 13': 'Emotions',
    'Topic 14': 'Experssions',
    'Topic 15': 'Dance',
    'Topic 16': 'Dreams',
    'Topic 17': 'Chance & Self',
    'Topic 18': 'Everyday Life',
    'Topic 19': 'Feelings',
    'Topic 20': 'Women'
}

topic_name_list = list(topic_names.values())

# Create a visualization
vis_data = pyLDAvis.gensim.prepare(lda_model, gensim_corpus, dictionary)
pyLDAvis.display(vis_data)


In [30]:
song_topic_distribution[0][0][0]

columns = ['Topic {}'.format(i + 1) for i in range(num_topics)]
song_topic_df = pd.DataFrame(columns=columns)

In [31]:
temp = df_cleaned[['Rank',	'Artist'	,'Song'	,'Genre',	'Year',	'Decade']].reset_index(drop=True)
temp

,Rank,Artist,Song,Genre,Year,Decade
0,1,Bob Dylan,Like a Rolling Stone,"Folk Rock,Singer/Songwriter",1965,1960
1,2,Nirvana,Smells Like Teen Spirit,Grunge,1991,1990
2,6,Aretha Franklin,Respect,Southern Soul,1967,1960
3,8,Marvin Gaye,What's Going On,Soul,1971,1970
4,9,Marvin Gaye,I Heard It Through the Grapevine,Motown Sound,1968,1960
...,...,...,...,...,...,...
2024,3988,Sean Paul,Get Busy,Ragga,2002,2000
2025,3992,PJ Harvey,Dress,"Alternative Rock,Singer/Songwriter",1991,1990
2026,3993,Jackson C. Frank,Blues Run the Game,"Contemporary Folk,Singer/Songwriter",1965,1960
2027,3997,Cat Stevens,Wild World,"Folk Pop,Singer/Songwriter",1970,1970


In [32]:
rows = []

# Populate the list with song-topic distribution data
for song_topics in song_topic_distribution:
    row_data = {f'Topic {topic_id + 1}': topic_prob for topic_id, topic_prob in song_topics}
    rows.append(row_data)

# Create the DataFrame from the list of dictionaries
song_topic_df = pd.DataFrame(rows)


In [33]:
topic_df = pd.concat([temp, song_topic_df], axis=1)
topic_df

,Rank,Artist,Song,Genre,Year,Decade,Topic 1,Topic 2,Topic 7,Topic 9,...,Topic 3,Topic 5,Topic 19,Topic 13,Topic 8,Topic 17,Topic 11,Topic 16,Topic 6,Topic 4
0,1,Bob Dylan,Like a Rolling Stone,"Folk Rock,Singer/Songwriter",1965,1960,0.031717,0.019127,0.085838,0.013023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Nirvana,Smells Like Teen Spirit,Grunge,1991,1990,NaN,0.109302,0.085454,NaN,...,0.080543,0.012526,0.406216,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,Aretha Franklin,Respect,Southern Soul,1967,1960,0.011331,0.013826,NaN,NaN,...,NaN,NaN,0.046755,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,Marvin Gaye,What's Going On,Soul,1971,1970,0.151443,NaN,NaN,0.040944,...,NaN,NaN,NaN,0.151561,NaN,NaN,NaN,NaN,NaN,NaN
4,9,Marvin Gaye,I Heard It Through the Grapevine,Motown Sound,1968,1960,0.071210,NaN,NaN,NaN,...,NaN,NaN,NaN,0.104584,0.057750,0.109226,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,3988,Sean Paul,Get Busy,Ragga,2002,2000,NaN,NaN,NaN,NaN,...,NaN,NaN,0.013347,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025,3992,PJ Harvey,Dress,"Alternative Rock,Singer/Songwriter",1991,1990,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.96182,NaN,NaN,NaN
2026,3993,Jackson C. Frank,Blues Run the Game,"Contemporary Folk,Singer/Songwriter",1965,1960,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.566602,NaN,NaN,NaN,0.422148,NaN
2027,3997,Cat Stevens,Wild World,"Folk Pop,Singer/Songwriter",1970,1970,NaN,0.011146,0.979670,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
def get_top_n_topics(df, n=3):
    # Exclude the 'Song' column from the calculations
    df_cols = df[['Rank','Artist','Song', 'Genre','Year','Decade']]
    
    # Create a list to store the DataFrames for each song
    results = []
    
    # Iterate through each row in the original DataFrame
    for index, row in df.iterrows():
        # Get the current song name
        song = row['Song']
        
        # Get the topics and their values for the current song
        song_topics = row[6:]
        
        # Convert the columns to numeric (ignore 'NaN' values)
        song_topics = pd.to_numeric(song_topics, errors='coerce')
        
        # Find the top n topics for the song
        top_topics = song_topics.nlargest(n)
        
        # Create a DataFrame for the current song
        # song_df = pd.DataFrame({
        #     'Song': [song],
        #     **{f'Top_{i}_Topic': [topic] for i, (topic, value) in enumerate(top_topics.items(), 1)}
        song_df = pd.DataFrame({
            **{f'Top_{i}_Topic': [topic] for i, (topic, value) in enumerate(top_topics.items(), 1)}
        })
        
        results.append(song_df)
    
    # Concatenate the results into a single DataFrame
    top_topics_df = pd.concat(results, ignore_index=True)
    top_topics_df = pd.concat([df_cols, top_topics_df], axis=1)
    
    return top_topics_df

# Usage:
top_3_topics_df = get_top_n_topics(topic_df, n=3)
top_3_topics_df


,Rank,Artist,Song,Genre,Year,Decade,Top_1_Topic,Top_2_Topic,Top_3_Topic
0,1,Bob Dylan,Like a Rolling Stone,"Folk Rock,Singer/Songwriter",1965,1960,Topic 18,Topic 14,Topic 10
1,2,Nirvana,Smells Like Teen Spirit,Grunge,1991,1990,Topic 19,Topic 18,Topic 2
2,6,Aretha Franklin,Respect,Southern Soul,1967,1960,Topic 18,Topic 19,Topic 2
3,8,Marvin Gaye,What's Going On,Soul,1971,1970,Topic 18,Topic 14,Topic 13
4,9,Marvin Gaye,I Heard It Through the Grapevine,Motown Sound,1968,1960,Topic 10,Topic 18,Topic 12
...,...,...,...,...,...,...,...,...,...
2024,3988,Sean Paul,Get Busy,Ragga,2002,2000,Topic 15,Topic 19,Topic 1
2025,3992,PJ Harvey,Dress,"Alternative Rock,Singer/Songwriter",1991,1990,Topic 11,Topic 20,Topic 1
2026,3993,Jackson C. Frank,Blues Run the Game,"Contemporary Folk,Singer/Songwriter",1965,1960,Topic 8,Topic 6,Topic 1
2027,3997,Cat Stevens,Wild World,"Folk Pop,Singer/Songwriter",1970,1970,Topic 7,Topic 2,Topic 1


In [35]:
# Mapping dictionary for the topic names
topic_mapping = {
    'Topic 1': 'Love',
    'Topic 2': 'Moments',
    'Topic 3': 'Nightlife',
    'Topic 4': 'Struggles',
    'Topic 5': 'Memories',
    'Topic 6': 'Aspiration',
    'Topic 7': 'World',
    'Topic 8': 'Identity',
    'Topic 9': 'Rock',
    'Topic 10': 'Relationships',
    'Topic 11': 'Desires',
    'Topic 12': 'Reality',
    'Topic 13': 'Emotions',
    'Topic 14': 'Expressions',
    'Topic 15': 'Dance',
    'Topic 16': 'Dreams',
    'Topic 17': 'Chance & Self',
    'Topic 18': 'Everyday Life',
    'Topic 19': 'Feelings',
    'Topic 20': 'Women'
}



# Replace the values in the 'Topic' column with the new topic names
top_3_topics_df['Top_1_Topic'] = top_3_topics_df['Top_1_Topic'].replace(topic_mapping)
top_3_topics_df['Top_2_Topic'] = top_3_topics_df['Top_2_Topic'].replace(topic_mapping)
top_3_topics_df['Top_3_Topic'] = top_3_topics_df['Top_3_Topic'].replace(topic_mapping)

In [36]:
top_3_topics_df

,Rank,Artist,Song,Genre,Year,Decade,Top_1_Topic,Top_2_Topic,Top_3_Topic
0,1,Bob Dylan,Like a Rolling Stone,"Folk Rock,Singer/Songwriter",1965,1960,Everyday Life,Expressions,Relationships
1,2,Nirvana,Smells Like Teen Spirit,Grunge,1991,1990,Feelings,Everyday Life,Moments
2,6,Aretha Franklin,Respect,Southern Soul,1967,1960,Everyday Life,Feelings,Moments
3,8,Marvin Gaye,What's Going On,Soul,1971,1970,Everyday Life,Expressions,Emotions
4,9,Marvin Gaye,I Heard It Through the Grapevine,Motown Sound,1968,1960,Relationships,Everyday Life,Reality
...,...,...,...,...,...,...,...,...,...
2024,3988,Sean Paul,Get Busy,Ragga,2002,2000,Dance,Feelings,Love
2025,3992,PJ Harvey,Dress,"Alternative Rock,Singer/Songwriter",1991,1990,Desires,Women,Love
2026,3993,Jackson C. Frank,Blues Run the Game,"Contemporary Folk,Singer/Songwriter",1965,1960,Identity,Aspiration,Love
2027,3997,Cat Stevens,Wild World,"Folk Pop,Singer/Songwriter",1970,1970,World,Moments,Love


In [37]:
from nrclex import NRCLex

# Analyze emotions using NRCLex
text = [NRCLex(lyric) for lyric in df_cleaned['Lyrics_Clean']]

# Get emotion counts
emotion_counts = [text.affect_frequencies for text in text]

In [38]:
emotion_df = pd.DataFrame(emotion_counts)

In [39]:
emotion_df.drop(columns=['anticip','positive','negative'], inplace = True)

In [40]:
def top_3_columns(row):
    top_cols = row.nlargest(3).index
    return ', '.join(top_cols)

# Create a new column for the top 3 columns in each row
emotion_df['top_3_columns'] = emotion_df.apply(top_3_columns, axis=1)

In [41]:
# Split 'top_3_columns' into separate columns
top_3_emotions = emotion_df['top_3_columns'].str.split(', ', expand=True)

# Rename the columns for clarity
top_3_emotions.columns = ['1st Emotion', '2nd Emotion', '3rd Emotion']

# Concatenate the original DataFrame with the new columns
emotion_df = pd.concat([emotion_df, top_3_emotions], axis=1)

# Drop the original 'top_3_columns' column if needed
emotion_df = emotion_df.drop(columns=['top_3_columns'])

# Display the result
emotion_df

,fear,anger,trust,surprise,sadness,disgust,joy,anticipation,1st Emotion,2nd Emotion,3rd Emotion
0,0.092784,0.072165,0.103093,0.061856,0.061856,0.041237,0.092784,0.144330,anticipation,trust,fear
1,0.100000,0.050000,0.037500,0.050000,0.025000,0.100000,0.137500,0.025000,joy,fear,disgust
2,0.015385,0.046154,0.153846,0.046154,0.015385,0.000000,0.230769,0.153846,joy,trust,anticipation
3,0.132743,0.097345,0.106195,0.000000,0.079646,0.053097,0.088496,0.079646,fear,trust,anger
4,0.072727,0.072727,0.036364,0.072727,0.090909,0.054545,0.163636,0.018182,joy,sadness,fear
...,...,...,...,...,...,...,...,...,...,...,...
2024,0.072289,0.012048,0.144578,0.120482,0.036145,0.000000,0.120482,0.192771,anticipation,trust,surprise
2025,0.062500,0.093750,0.078125,0.046875,0.125000,0.125000,0.140625,0.062500,joy,sadness,disgust
2026,0.140351,0.070175,0.000000,0.035088,0.210526,0.070175,0.105263,0.017544,sadness,fear,joy
2027,0.088083,0.041451,0.056995,0.108808,0.088083,0.041451,0.170984,0.077720,joy,surprise,fear


In [42]:
# top_topics_df = pd.concat([df_cols, top_topics_df], axis=1)
top_emotions_df = pd.concat( [temp.iloc[:, :6],emotion_df.iloc[:, -3:]],axis = 1)

In [43]:
top_emotions_df

,Rank,Artist,Song,Genre,Year,Decade,1st Emotion,2nd Emotion,3rd Emotion
0,1,Bob Dylan,Like a Rolling Stone,"Folk Rock,Singer/Songwriter",1965,1960,anticipation,trust,fear
1,2,Nirvana,Smells Like Teen Spirit,Grunge,1991,1990,joy,fear,disgust
2,6,Aretha Franklin,Respect,Southern Soul,1967,1960,joy,trust,anticipation
3,8,Marvin Gaye,What's Going On,Soul,1971,1970,fear,trust,anger
4,9,Marvin Gaye,I Heard It Through the Grapevine,Motown Sound,1968,1960,joy,sadness,fear
...,...,...,...,...,...,...,...,...,...
2024,3988,Sean Paul,Get Busy,Ragga,2002,2000,anticipation,trust,surprise
2025,3992,PJ Harvey,Dress,"Alternative Rock,Singer/Songwriter",1991,1990,joy,sadness,disgust
2026,3993,Jackson C. Frank,Blues Run the Game,"Contemporary Folk,Singer/Songwriter",1965,1960,sadness,fear,joy
2027,3997,Cat Stevens,Wild World,"Folk Pop,Singer/Songwriter",1970,1970,joy,surprise,fear


In [44]:
sid = SentimentIntensityAnalyzer()
def nltk_sentiment_score(text):
    # Get the polarity scores for the text
    scores = sid.polarity_scores(text)
    return scores['pos'], scores['neg'], scores['neu'], scores['compound']

In [45]:
# Applying nltk_sentiment_score to 'Lyrics_Clean' column
sentiment_scores = df_cleaned['Lyrics_Clean'].apply(nltk_sentiment_score)

# Unpacking the tuple of scores into separate Series
sentiment_pos, sentiment_neg, sentiment_neu, sentiment_comp = zip(*sentiment_scores)

# Creating new columns in the DataFrame
df_cleaned['sentiment_pos'] = sentiment_pos
df_cleaned['sentiment_neg'] = sentiment_neg
df_cleaned['sentiment_neu'] = sentiment_neu
df_cleaned['sentiment_comp'] = sentiment_comp

In [46]:
data_sentiment = {
    'Positive': sentiment_pos,
    'Negative': sentiment_neg,
    'Neutral': sentiment_neu
    # 'sentiment_comp': sentiment_comp
}

test = pd.DataFrame(data_sentiment)
test

top_sentiments_df = pd.concat( [temp.iloc[:, :4],test],axis = 1)


In [47]:
top_sentiments_df.dtypes

Rank          int64
Artist       object
Song         object
Genre        object
Positive    float64
Negative    float64
Neutral     float64
dtype: object

In [48]:
def determine_sentiment(row):
    # highest_sentiment = row[['Positive', 'Negative', 'Neutral']].idxmax()
    # return highest_sentiment
    max_sentiment = max(row['Positive'], row['Negative'])#, row['Neutral'])
    if max_sentiment == row['Positive']:
        return 'Positive'
    elif max_sentiment == row['Negative']:
        return 'Negative'
    else:
        return 'Neutral'

# Create a new column 'Sentiment'
top_sentiments_df['Sentiment'] = top_sentiments_df.apply(determine_sentiment, axis=1)

In [49]:
top_sentiments_df = pd.concat( [temp.iloc[:, :6],top_sentiments_df.iloc[:, -1:]],axis = 1)
sentiments_df = pd.DataFrame(top_sentiments_df.groupby('Decade')['Sentiment'].value_counts())
sentiments_df
sentiments_df.to_csv('sentiments_df.csv')

In [50]:
top_emotions_df

,Rank,Artist,Song,Genre,Year,Decade,1st Emotion,2nd Emotion,3rd Emotion
0,1,Bob Dylan,Like a Rolling Stone,"Folk Rock,Singer/Songwriter",1965,1960,anticipation,trust,fear
1,2,Nirvana,Smells Like Teen Spirit,Grunge,1991,1990,joy,fear,disgust
2,6,Aretha Franklin,Respect,Southern Soul,1967,1960,joy,trust,anticipation
3,8,Marvin Gaye,What's Going On,Soul,1971,1970,fear,trust,anger
4,9,Marvin Gaye,I Heard It Through the Grapevine,Motown Sound,1968,1960,joy,sadness,fear
...,...,...,...,...,...,...,...,...,...
2024,3988,Sean Paul,Get Busy,Ragga,2002,2000,anticipation,trust,surprise
2025,3992,PJ Harvey,Dress,"Alternative Rock,Singer/Songwriter",1991,1990,joy,sadness,disgust
2026,3993,Jackson C. Frank,Blues Run the Game,"Contemporary Folk,Singer/Songwriter",1965,1960,sadness,fear,joy
2027,3997,Cat Stevens,Wild World,"Folk Pop,Singer/Songwriter",1970,1970,joy,surprise,fear


In [51]:
Emotion1 = pd.DataFrame(top_emotions_df.groupby('Decade')['1st Emotion'].value_counts())
Emotion1

count
Decade 1st Emotion        
1960   joy              96
       fear             40
       sadness          38
       trust            37
       anticipation     29
       anger            18
       surprise          5
       disgust           4
1970   joy             142
       fear             83
       trust            80
       anticipation     78
       sadness          60
       anger            19
       surprise         15
       disgust          10
1980   joy              89
       fear             72
       anticipation     54
       trust            51
       sadness          44
       anger            30
       disgust           6
       surprise          4
1990   fear             84
       joy              79
       anticipation     61
       sadness          56
       trust            37
       anger            33
       disgust           8
       surprise          6
2000   fear             58
       anticipation     55
       joy              41
       sadness          40
       trust            35
       anger            24
       disgust           9
       surprise          5
2010   joy              60
       fear             57
       sadness          53
       anticipation     44
       trust            38
       anger            30
       surprise          7
       disgust           5

In [52]:
Emotion2 = pd.DataFrame(top_emotions_df.groupby('Decade')['2nd Emotion'].value_counts())
Emotion2

count
Decade 2nd Emotion        
1960   trust            62
       fear             45
       sadness          44
       anticipation     39
       joy              39
       anger            23
       surprise         10
       disgust           5
1970   trust           102
       fear             81
       anticipation     77
       joy              71
       sadness          61
       anger            57
       surprise         28
       disgust          10
1980   fear             68
       trust            62
       anticipation     56
       joy              50
       anger            48
       sadness          44
       surprise         15
       disgust           7
1990   fear             71
       trust            70
       sadness          65
       anger            50
       anticipation     44
       joy              39
       disgust          15
       surprise         10
2000   fear             49
       trust            44
       joy              42
       sadness          40
       anger            37
       anticipation     30
       surprise         15
       disgust          10
2010   fear             58
       sadness          55
       joy              44
       trust            42
       anticipation     35
       anger            28
       disgust          18
       surprise         14

In [53]:
Emotion3 = pd.DataFrame(top_emotions_df.groupby('Decade')['3rd Emotion'].value_counts())
Emotion3


count
Decade 3rd Emotion        
1960   anticipation     51
       trust            41
       fear             39
       sadness          36
       anger            34
       joy              27
       surprise         26
       disgust          13
1970   trust            82
       sadness          75
       anticipation     74
       anger            66
       fear             66
       joy              60
       surprise         40
       disgust          24
1980   trust            61
       anticipation     58
       sadness          53
       anger            51
       joy              48
       fear             45
       surprise         24
       disgust          10
1990   trust            59
       sadness          53
       anger            52
       fear             50
       joy              50
       anticipation     46
       disgust          29
       surprise         25
2000   trust            43
       sadness          42
       anticipation     42
       joy              37
       fear             36
       anger            35
       surprise         23
       disgust           9
2010   anger            55
       fear             54
       sadness          37
       trust            37
       joy              35
       anticipation     31
       surprise         25
       disgust          20

In [54]:
top_3_topics_df

,Rank,Artist,Song,Genre,Year,Decade,Top_1_Topic,Top_2_Topic,Top_3_Topic
0,1,Bob Dylan,Like a Rolling Stone,"Folk Rock,Singer/Songwriter",1965,1960,Everyday Life,Expressions,Relationships
1,2,Nirvana,Smells Like Teen Spirit,Grunge,1991,1990,Feelings,Everyday Life,Moments
2,6,Aretha Franklin,Respect,Southern Soul,1967,1960,Everyday Life,Feelings,Moments
3,8,Marvin Gaye,What's Going On,Soul,1971,1970,Everyday Life,Expressions,Emotions
4,9,Marvin Gaye,I Heard It Through the Grapevine,Motown Sound,1968,1960,Relationships,Everyday Life,Reality
...,...,...,...,...,...,...,...,...,...
2024,3988,Sean Paul,Get Busy,Ragga,2002,2000,Dance,Feelings,Love
2025,3992,PJ Harvey,Dress,"Alternative Rock,Singer/Songwriter",1991,1990,Desires,Women,Love
2026,3993,Jackson C. Frank,Blues Run the Game,"Contemporary Folk,Singer/Songwriter",1965,1960,Identity,Aspiration,Love
2027,3997,Cat Stevens,Wild World,"Folk Pop,Singer/Songwriter",1970,1970,World,Moments,Love
